In [1]:
# ungzip! file
! gunzip 2015_07_22_mktplace_shop_web_log_sample.log.gz

gzip: 2015_07_22_mktplace_shop_web_log_sample.log.gz: No such file or directory


In [2]:
! head 2015_07_22_mktplace_shop_web_log_sample.log
! wc -l 2015_07_22_mktplace_shop_web_log_sample.log

2015-07-22T09:00:28.019143Z marketpalce-shop 123.242.248.130:54635 10.0.6.158:80 0.000022 0.026109 0.00002 200 200 0 699 "GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36" ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2
2015-07-22T09:00:27.894580Z marketpalce-shop 203.91.211.44:51402 10.0.4.150:80 0.000024 0.15334 0.000026 200 200 0 1497 "GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20100101 Firefox/39.0" ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2
2015-07-22T09:00:27.885745Z marketpalce-shop 1.39.32.179:56419 10.0.4.244:80 0.000024 0.164958 0.000017 200 200 0 157 "GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like G

In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().set("spark.executor.memory", "4g").setMaster('local[20]').setAppName('WebLogAnalysis')
sc = SparkContext(conf = conf)

# import spark.sql
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [2]:
from pathlib import Path
CURRENT_DIR = Path('.')
DATA_PATH = CURRENT_DIR / '2015_07_22_mktplace_shop_web_log_sample.log'

In [3]:
raw_text_file = sc.textFile(DATA_PATH.absolute().as_posix())

In [4]:
! pip install python-dateutil


In [4]:
column_names = ['timestamp', 'elb', 'client_port', 'backend_port', 'request_processing_time',
               'backend_processing_time', 'response_processing_time', 'elb_status_code',
                'backend_status_code', 'received_bytes', 'sent_bytes', 'request',
                'user_agent', 'ssl_cipher', 'ssl_protocol'
               ]
colname_to_idx = dict(zip(column_names, range(len(column_names))))

from pyspark.sql.types import *

schema = StructType([
    StructField('timestamp', TimestampType(), False),
    StructField('elb', StringType(), False),
    StructField('client_port', StringType(), False),
    StructField('backend_port', StringType(), False),
    StructField('request_processing_time', DoubleType(), False),
    StructField('backend_processing_time', DoubleType(), False),
    StructField('response_processing_time', DoubleType(), False),
    StructField('elb_status_code', StringType(), False),
    StructField('backend_status_code', StringType(), False),
    StructField('received_bytes', IntegerType(), False),
    StructField('sent_bytes', IntegerType(), False),
    StructField('request', StringType(), False),
    StructField('user_agent', StringType(), False),
    StructField('ssl_cipher', StringType(), False),
    StructField('ssl_protocol', StringType(), False)
])


In [5]:
import dateutil.parser
import re

def process_log_line(line):

    pattern = r'^(.*?)"(.*?)" "(.*?)"(.*?)$'
    parts = re.findall(pattern,line)[0]
    all_data_fields = parts[0].strip().split() + [parts[1].strip()] + [parts[2].strip()] + parts[3].strip().split()

    # convert timestamp to datetime
    all_data_fields[colname_to_idx['timestamp']] = dateutil.parser.parse(all_data_fields[colname_to_idx['timestamp']])
    # convert request_processing_time, backend_processing_time, response_processing_time to float
    all_data_fields[colname_to_idx['request_processing_time']] = float(all_data_fields[colname_to_idx['request_processing_time']])
    all_data_fields[colname_to_idx['backend_processing_time']] = float(all_data_fields[colname_to_idx['backend_processing_time']])
    all_data_fields[colname_to_idx['response_processing_time']] = float(all_data_fields[colname_to_idx['response_processing_time']])
    # convert bytes to integer
    all_data_fields[colname_to_idx['received_bytes']] = int(all_data_fields[colname_to_idx['received_bytes']])
    all_data_fields[colname_to_idx['sent_bytes']] = int(all_data_fields[colname_to_idx['sent_bytes']])    
    
    return all_data_fields


In [6]:
parts = raw_text_file.map(lambda line: process_log_line(line))

In [7]:
df = spark.createDataFrame(parts, schema=schema)

In [8]:
df.head()

Py4JJavaError: An error occurred while calling o40.collectToPython.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/jovyan/work/experiments/2015_07_22_mktplace_shop_web_log_sample.log
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions

In [10]:
import pyspark.sql.functions as F

In [11]:
# get IP from 'client:port'
fn = F.udf(lambda x:x.split(':')[0], StringType())
df2 = df.withColumn('IP', fn(df.client_port))

In [12]:
df2.head()

Row(timestamp=datetime.datetime(2015, 7, 22, 9, 0, 28, 19143), elb='marketpalce-shop', client_port='123.242.248.130:54635', backend_port='10.0.6.158:80', request_processing_time=2.2e-05, backend_processing_time=0.026109, response_processing_time=2e-05, elb_status_code='200', backend_status_code='200', received_bytes=0, sent_bytes=699, request='GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1', user_agent='Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36', ssl_cipher='ECDHE-RSA-AES128-GCM-SHA256', ssl_protocol='TLSv1.2', IP='123.242.248.130')

In [144]:
df3 = df2.select('IP', 'timestamp')

In [145]:
%%time

x = df3.groupBy('IP').count().orderBy('count', ascending=False).collect()

CPU times: user 295 ms, sys: 42.2 ms, total: 337 ms
Wall time: 39.4 s


In [146]:
import pyspark.sql as pysql
import pyspark.sql.functions as F

In [150]:
timewindow = pysql.Window.partitionBy('IP').orderBy('timestamp')
y = df3.withColumn('prevtime', F.lag('timestamp',1).over(timewindow))

d = y.withColumn('diff',\
                 F.when(F.isnull(F.unix_timestamp(y.timestamp)-F.unix_timestamp(y.prevtime)), 0)\
                 .otherwise(F.unix_timestamp(y.timestamp)-F.unix_timestamp(y.prevtime)))
s = d.withColumn('is_new_session', F.when(d.diff > 15*60, 1).otherwise(0))

new_sess_window = pysql.Window.partitionBy('IP').orderBy('timestamp').rowsBetween(pysql.Window.unboundedPreceding,pysql.Window.currentRow)
ss = s.withColumn('session_id', F.sum('is_new_session').over(new_sess_window))

In [151]:
ss.orderBy('diff',ascending=False).show(1000,truncate=False)

+---------------+--------------------------+--------------------------+-----+--------------+----------+
|IP             |timestamp                 |prevtime                  |diff |is_new_session|session_id|
+---------------+--------------------------+--------------------------+-----+--------------+----------+
|27.120.106.3   |2015-07-22 21:07:43.815142|2015-07-22 02:42:44.897384|66299|1             |1         |
|117.255.253.155|2015-07-22 21:07:17.319098|2015-07-22 05:10:14.538692|57423|1             |1         |
|103.24.125.26  |2015-07-22 21:08:49.872864|2015-07-22 05:14:03.816461|57286|1             |1         |
|98.230.153.173 |2015-07-22 21:07:50.050525|2015-07-22 05:13:41.422524|57249|1             |1         |
|150.228.40.140 |2015-07-22 21:06:09.760099|2015-07-22 05:12:35.964285|57214|1             |1         |
|14.139.69.64   |2015-07-22 21:06:37.132369|2015-07-22 05:13:45.485184|57172|1             |1         |
|66.249.82.186  |2015-07-22 21:05:52.421404|2015-07-22 05:13:45.

In [152]:
ss.where(ss.IP=='202.91.134.8').show(10000,truncate=False)

+------------+--------------------------+--------------------------+-----+--------------+----------+
|IP          |timestamp                 |prevtime                  |diff |is_new_session|session_id|
+------------+--------------------------+--------------------------+-----+--------------+----------+
|202.91.134.8|2015-07-22 05:13:44.826558|null                      |0    |0             |0         |
|202.91.134.8|2015-07-22 05:13:45.693475|2015-07-22 05:13:44.826558|1    |0             |0         |
|202.91.134.8|2015-07-22 05:13:45.804754|2015-07-22 05:13:45.693475|0    |0             |0         |
|202.91.134.8|2015-07-22 05:14:18.767052|2015-07-22 05:13:45.804754|33   |0             |0         |
|202.91.134.8|2015-07-22 05:14:18.930119|2015-07-22 05:14:18.767052|0    |0             |0         |
|202.91.134.8|2015-07-22 05:14:44.204098|2015-07-22 05:14:18.930119|26   |0             |0         |
|202.91.134.8|2015-07-22 05:14:46.041637|2015-07-22 05:14:44.204098|2    |0             |0 